In [1]:
import tomotopy as tp
import numpy as np
import csv
import nltk
import string
from sklearn import metrics
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from dateutil import parser
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sklearn.cluster as sc
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
nltk.download('stopwords')
s=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorenchamplin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def lowercase(input):
  """
  Returns lowercase text
  """
  return input.lower()

def remove_punctuation(input):
  """
  Returns text without punctuation
  """
  return input.translate(str.maketrans('','', string.punctuation+'–’”“—·'))

def remove_whitespaces(input):
  """
  Returns text without extra whitespaces
  """
  return " ".join(input.split())
  
def remove_html_tags(input):
  """
  Returns text without HTML tags
  """
  soup = BeautifulSoup(input, "html.parser")
  stripped_input = soup.get_text(separator=" ")
  return stripped_input

def tokenize(input):
  """
  Returns tokenized version of text
  """
  return word_tokenize(input)

def remove_stop_words(input):
  """
  Returns text without stop words
  """
  input = word_tokenize(input)
  return [word for word in input if word not in stopwords.words('english') or word == "no" or word == "not"]

def lemmatize(input):
  """
  Lemmatizes input using NLTK's WordNetLemmatizer
  """
  lemmatizer=WordNetLemmatizer()
  input_str=word_tokenize(input)
  new_words = []
  for word in input_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)


def nlp_pipeline(input):
  """
  Function that calls all other functions together to perform NLP on a given text
  """
  return lemmatize(' '.join(remove_stop_words(remove_whitespaces(remove_punctuation(remove_html_tags(lowercase(input)))))))

In [3]:
mdl = tp.LDAModel(tw=tp.TermWeight.ONE,k=6,seed=2021)
data = []
sentences = []
truth_labels = []
with open("../comments.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        truth_labels.append(row[1])
        sentences.append(row[0])
        line = nlp_pipeline(row[0]).split()
        if line:
            mdl.add_doc(nlp_pipeline(row[0]).split())
            data.append(nlp_pipeline(row[0]).split())

In [4]:
for i in range(0, 500, 10):
    mdl.train(10)

mdl.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 694 docs, 21405 words
| Total Vocabs: 3117, Used Vocabs: 3117
| Entropy of words: 6.48915
| Entropy of term-weighted words: 6.48915
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -6.96409
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 6 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2021 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.2959824  0.20503

In [5]:
samples = []
for i in data:
    samples.append(mdl.make_doc(i))

In [6]:
X = mdl.infer(samples)

In [7]:
def cluster_and_evaluate(texts, X,truth_labels):
    clustering = sc.AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=0.1,
        affinity='cosine',
        linkage='average')
    clustering.fit(X)
    print("\n")
    print("=== Unsupervised Metrics ===\n")
    print("== Silhouette Coefficient [0,1*]==")
    print(metrics.silhouette_score(X, clustering.labels_, metric='cosine'),"\n")
    print("== Calinski-Harabasz Index [0,inf*] ==")
    print(metrics.calinski_harabasz_score(X, clustering.labels_),"\n")
    print("== Davies-Bouldin Index [0*,1] ==")
    print(metrics.davies_bouldin_score(X, clustering.labels_),"\n")
    print("\n")
    print("=== Supervised Metrics ===\n")
    print("== Rand Index [0,1*] ==")
    print(metrics.rand_score(truth_labels, clustering.labels_),"\n")
    print("== Normalized Mutual Information Score [0,1*] ==")
    print(metrics.normalized_mutual_info_score(truth_labels, clustering.labels_),"\n")
    print("== Fowlkes-Mallows Score [0,1*] ==")
    print(metrics.fowlkes_mallows_score(truth_labels, clustering.labels_),"\n")

In [8]:
X = np.array(X[0])

In [9]:
cluster_and_evaluate(sentences, X,truth_labels)



=== Unsupervised Metrics ===

== Silhouette Coefficient [0,1*]==
0.51889884 

== Calinski-Harabasz Index [0,inf*] ==
158.85115380748968 

== Davies-Bouldin Index [0*,1] ==
0.8832133511288746 



=== Supervised Metrics ===

== Rand Index [0,1*] ==
0.5372123873564796 

== Normalized Mutual Information Score [0,1*] ==
0.26700745506587076 

== Fowlkes-Mallows Score [0,1*] ==
0.4048308619059552 

